#Consegna
- **Implementare l’algoritmo di Lesk**
1. Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di
WN) e disambiguare (almeno) un sostantivo per frase. Calcolare
l’accuratezza del sistema implementato, sulla base dei sensi annotati in
SemCor.
2. Randomizzare la selezione delle 50 frasi e la selezione del termine da
disambiguare, e restituire l’accuratezza media su (per esempio) 10
esecuzioni del programma.

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('semcor')
nltk.download('wordnet')
from nltk.tree import Tree
import statistics

from nltk.corpus import wordnet as wn
from nltk.corpus import semcor
from nltk.stem import WordNetLemmatizer 
from xml.dom import minidom
import random
from nltk.corpus.reader.wordnet import Lemma

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data]   Package semcor is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Funzioni ausiliarie

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words_list = []

# Ritorna il POS TAG di una parola
def get_wordnet_pos(word):
    treebank_tag = [tag for (word, tag) in nltk.pos_tag(nltk.word_tokenize(word))][0]
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return ''

# Ritorna una lista con le stop words
def get_stop_words() :
    if len(stop_words_list) == 0 :
        f = open("utils/stop_words_FULL.txt", "r")
        for x in f:
            stop_words_list.append(x)

    return stop_words_list

# Da una frase ritorna una lista con le singole parole (lemmi) rimuovendo le parole inutili (stop words..)
def get_list_of_gains_words(sentence) :
    list_words_lemma = []
    aus_list_words = sentence.split()
    stop_words_list = get_stop_words()

    for w in aus_list_words :
        if w.lower() not in stop_words_list :
            pos_tag = get_wordnet_pos(w)
            if pos_tag != '' :
                list_words_lemma.append(lemmatizer.lemmatize(w.lower(), pos_tag))

    return list_words_lemma

# Ritorna la lista delle parole della signature di un synset : gloss + examples
def get_signature_of_synset(synset) :
    # Gloss
    signature = get_list_of_gains_words(synset.definition())
    # Examples
    for example in synset.examples() :
        list_ex = get_list_of_gains_words(example)
        signature.extend(list_ex)

    return signature

# Calcola il numero di elementi comuni di due liste
def overlap_lists(list1, list2) :
    common_elements = set(list1) & set(list2)
    return len(common_elements)

#Algoritmo Lesk 

In [ ]:

"""
    Lesk Algorithm che va a disambiguare una parola dato un contesto
    :return: il miglior synset disambiguato della parola nel contesto
"""
def lesk_algorithm(word, sentence) :
    #prendo il primo synset della parola
    word_best_sense = wn.synsets(word)[0]
    max_overlap = 0

    #prendiamo il contesto della parola, ossia la frase senza le parole inutili
    context = get_list_of_gains_words(sentence)
    #per ogni senso della parola
    for current_sense in wn.synsets(word) :
        #Prendo la Signature
        signature_sense = get_signature_of_synset(current_sense)
        # Se la signature è piccola, prendo le info anche dagli iponimi
        if len(signature_sense) < 25 : 
            #Aggiungo le info degli iponomi
            for hypo in current_sense.hyponyms() :
                signature_sense.extend(get_signature_of_synset(hypo))  
              
        #Overlap: quante parole in comune tra il contesto e la signature del senso associato,
        #è quello che sto cercando
        overlap = overlap_lists(context, signature_sense)
        #mi interessa il miglior senso con l'overlap maggiore
        if overlap > max_overlap :
            max_overlap = overlap
            word_best_sense = current_sense

    return word_best_sense

#Lettura frasi da SemCore

In [ ]:
"""
   Quello che vogliamo fare è estrarre 50 frasii su un corpus di 100 ma in modo casuale
   :return: 
      -50 frasi casuali prese da Semcor 
      -parole polisemiche da disambiguare prese dalle 50 frasi
"""
def get_semcor_sentence() :
    semcor_sentences = []
    polysemic_words = []
    #Estrae 50 frasi in posizione casuale compresa tra 0 e 100
    for index in random.sample(range(0, 100), 50):
        elem = []
        #tagged_sents() the given file(s) as a list of semcor_sentences. Each sentence is represented as a list of tagged chunks (in tree form).
        #tags= both comprende sia il tag pos (part of speech) che il tag sem (semantic)
        for noun_tree in semcor.tagged_sents(tag='both')[index]:
            #se noun_tree.label è istanza di Lemma, è un nome (pos tag NN) e ha almeno un synset associato
            # Il controllo è passato solo da nomi che abbiano almeno un synset
            if isinstance(noun_tree.label(), Lemma) and noun_tree[0].label() == 'NN' and len(wn.synsets(noun_tree[0][0])) > 0:
                #lo aggiungo alla lista degli elementi
                elem.append(noun_tree)
        if elem :
            #Se elem non è vuota ma contiene le parole polisemiche la aggiungo alla lista 
            #polysemic_words randomizzando l'ordine con cui vengono messi i nomi
            polysemic_words.append(random.choice(elem))
            #E aggiungo anche la frase alla lista semcor_sentences
            semcor_sentences.append(" ".join(semcor.sents()[index]))
    return semcor_sentences, polysemic_words

#Main

In [ ]:
if __name__ == "__main__" :
    results=[]
    
    #Facciamo girare il programma 10 volte facendo una media dei valori ottenuti 
    for i in range(0,10):
        # Lesk Test
        semcor_sentences, polysemic_words = get_semcor_sentence()
        right_sense = 0
    
        #invochiamo il Lesk alorith per tutte le 50 frasi estratte da Semcor
        for i in range(0,50) :
            
            #invochiamo il lesk algoritmh passando la prima parola ambigua da disambiguare
            #polysemic_words[i])[0][0] significa prendi l'i-esimo albero contenuto in polysemic_words,
            # e di quetso albero (label, (pos, nome)) prendi il nome
            word_best_sense = lesk_algorithm((polysemic_words[i])[0][0],semcor_sentences[i])

            # Se il senso individuato dal lesk algorithm corrisponde a quello
            # presente su semcor allora incrementiamo il contatore dei risultati positivi "right_sense"
            if polysemic_words[i].label() in word_best_sense.lemmas() :
                right_sense+=1
            """
            else:
                print("Frase : " + semcor_sentences[i])
                print("Sostantivo : " + str((polysemic_words[i])[0][0]) + " -> " + str((polysemic_words[i])))
                print("Lesk : " + str(word_best_sense))
                print("######################################################")
            """

        print("#CORRECT : {}% ".format(float(right_sense/50)*100) + "({}/50)".format(str(right_sense)))
        results.append(float(right_sense/50)*100)
    
    print("La media di tutti i punteggi su 10 iterazioni è " + str(statistics.mean(results)))
    
    

LookupError: ignored

#Ouput 
#CORRECT : 88.0% (44/50)
#CORRECT : 90.0% (45/50)
#CORRECT : 90.0% (45/50)
#CORRECT : 86.0% (43/50)
#CORRECT : 86.0% (43/50)
#CORRECT : 86.0% (43/50)
#CORRECT : 92.0% (46/50)
#CORRECT : 94.0% (47/50)
#CORRECT : 92.0% (46/50)
#CORRECT : 92.0% (46/50)
La media di tutti i punteggi su 10 iterazioni è 89.6